In [ ]:
import requests
import numpy as np
import os, sys
import random
import math
import bisect
import arrow
import pytz
from datetime import datetime, date, timedelta
from dateutil import tz
from timezonefinder import TimezoneFinder
from matplotlib import pyplot as plt, dates
from matplotlib.ticker import *
from matplotlib_helper import *
from carbon_api_client import *
from typing import List

In [ ]:
enable_savefig = False

In [ ]:
def get_all_fuel_types(energy_mixture_timeseries):
    fuel_types = set()
    for entry in energy_mixture_timeseries:
        fuel_types.update(entry['power_by_fuel_type'].keys())
    return fuel_types

In [ ]:
def get_timeseries_by_fuel_type(energy_mixture_timeseries, fuel_types):
    d_timeseries_by_fuel_type = {}
    for fuel_type in fuel_types:
        d_timeseries_by_fuel_type[fuel_type] = []
    for entry in energy_mixture_timeseries:
        for fuel_type in fuel_types:
            value = entry['power_by_fuel_type'][fuel_type] if fuel_type in entry['power_by_fuel_type'] else 0.
            d_timeseries_by_fuel_type[fuel_type].append(value)
    return d_timeseries_by_fuel_type

In [ ]:
# Similar to the bar plot in supply trend in http://www.caiso.com/todaysoutlook/pages/supply.html

region = 'US-CAISO'
CA_GPS = (36.78, -119.78)

target_date = arrow.get(date(2022, 7, 3), tzinfo=tz.gettz('America/Los_Angeles'))
energy_mixture_timeseries = call_sysnet_energy_mixture_api(CA_GPS[0], CA_GPS[1], target_date, target_date.shift(days=1))
# print(energy_mixture_timeseries[:2])

timestamps = [e['timestamp'] for e in energy_mixture_timeseries]
# fuel_types = get_all_fuel_types(energy_mixture_timeseries)
fuel_types = { fuel_type for entry in energy_mixture_timeseries for fuel_type in entry['power_by_fuel_type'] }
d_fuel_type_timeseries = get_timeseries_by_fuel_type(energy_mixture_timeseries, fuel_types)

print('fuel_types', fuel_types)
# print('timestamps len:', len(timestamps))
# for fuel_type in d_fuel_type_timeseries:
#     print(fuel_type, 'len:', len(d_fuel_type_timeseries[fuel_type]))

plt.figure(figsize=(8, 4.8))
fuel_order = []
plot_colors = {
    'coal': 'black',
    'gas': '#F2915F',
    'nuclear': '#808080',
    'geothermal': 'brown',
    'biomass': 'yellowgreen',
    'hydro': '#40CDC4',
    'wind': 'royalblue',
    'solar': 'gold',
}
plot_stacked_line(timestamps, d_fuel_type_timeseries, order=plot_colors.keys(), d_label_colors=plot_colors)

plt.xlabel('Time')
plt.ylabel('Power (MW)')
plt.title(f'Energy in MW by fuel type in {region} on {target_date.strftime("%Y-%m-%d")}')
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.legend(loc='center left', bbox_to_anchor=(1.04, 0.5), ncol=1)
plt.xticks(rotation=15)
# plt.ylim(0, 1000)
plt.tight_layout()
savefig_filename = 'energy-by-fuel-mix.%s.%s.png' % (target_date.strftime("%Y-%m-%d"), region)
if enable_savefig:
    plt.savefig(savefig_filename)